In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from sympy import content


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie special list. if you got a question about movies. you should answer with only three emojis. dont use any words."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough().assign(history=load_memory) | prompt | chat


def invoke_chain(question):
    result = chain.invoke({"question" : question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


In [2]:
invoke_chain("Star wars.")

content='🌌👽⚔️'


In [3]:
invoke_chain("do you know star wars?")

content='🌌👽⚔️'
